### Funciones

In [98]:
def str_to_date(string):
  from datetime import datetime
  date = datetime.strptime(string[:-9], '%Y-%m-%d')  # Se toma únicamente las posiciones dd/mm/yyyy
  return date

def str_to_hour(string):
  from datetime import datetime
  date = datetime.strptime(string, '%Y-%m-%d %H:%M:%S')
  return date

def getHour(string):
  return int(string[:2])

def getHour_2(hora_hechos):
  hora = float(hora_hechos.hour)
  return hora

def getYear(timestamp):
  return int(timestamp.year)

def getMonth(timestamp):
  return timestamp.month

def getDay(timestamp):
  return timestamp.day

def getWeek(timestamp):
  return timestamp.week

def getWeek_Day_Name(timestamp):
  dias = ['LUNES', 'MARTES', 'MIÉRCOLES', 'JUEVES', 'VIERNES', 'SÁBADO', 'DOMINGO']
  return dias[timestamp.weekday()]

def get_semana_total(df_):
  semana_total_ = []

  for index,fecha in enumerate(df_.fecha_hecho):
    año = str(fecha.year)
    semana = str(fecha.week)
    cadena = str(año+'_'+semana)

    semana_total_.append(cadena)
    
  return semana_total_

def str_comma_to_float(string):
  string = str(string).replace(',','.')
  return float(string)

def fix_coordinates(string):
  return float(string[:-1])

def rango_edades(edad):
  if  0 <= edad <= 14:
    grupo_edad = 0
  elif 15 <= edad <= 18:
    grupo_edad = 1
  elif edad >= 19:
    grupo_edad = 2
  return grupo_edad

def fix_comuna_barrio(point,geojson,codigos=['codcom']):
  point = Point(point)
  for poly in geojson['features']:
    polygon = shape(poly['geometry'])
    if point.within(polygon):
      r_tuple = tuple([str(poly['properties'][codigo]) for codigo in codigos])
      return r_tuple
  return ('RURAL',)*len(codigos)

def comuna_id(string):
  if string == 'RURAL':
    ID = ''
  else:
    ID = int(string)
  return ID

def outside_drop(geo_pe, df):
  afuera = []
  for i in tqdm(range(len(df))):
    df.x = df.x.apply(float)
    df.y = df.y.apply(float)
    point = Point(df[['x','y']].iloc[i])
    adentro = False
    for poly in geo_pe['features']:
      polygon = shape(poly['geometry'])
      adentro += point.within(polygon)

    if not adentro:
      afuera.append(i)
  df = df.drop(afuera)
  print(len(afuera))
  return df

def str_to_bool(string):
  if 'SI' in string:
    out = 1
  else:
    out = 0
  return out

def bool_to_string(num):
  if num == 0:
    string = 'NO'
  else:
    string = 'SI'
  return string

def get_index(casper, caso):
  for indx,cas in enumerate(casper):
    if caso in cas:
      return indx

def get_ult_act_cas(df_,no_homicidio=True):
  new_cas = []   # Se van almacenando los caso_id que aparecen por primera vez
  cas_per = ['']   # Se almacena caso_id con todos los pers_id que le correspondan
  indx_to_drop = []

  for i in tqdm(reversed(range(len(df_)))):               # Se recorre el dataframe en reversa y se toman estas variables en cada fila
    caso_id_ = 'c'+str(df_.iloc[i]['caso_id'])
    if no_homicidio:
      pers_id_vin = 'p'+str(df_.iloc[i]['pers_id'])+'_'+str(df_.iloc[i]['tipo_vinculacion'])
    
    if i != np.max(range(len(df_))):                             # Aquí se cerrará elregistro más actual de cada caso
      cas_ant = 'c'+str(df_.iloc[i+1]['caso_id'])                # caso_id analizado justo antes
      casperant = cas_per[get_index(cas_per,cas_ant)]            # caso+personas que fue creado para el caso_id analizado justo antes
      if (caso_id_ != cas_ant) and ('_FIN' not in casperant):    # caso_id actual vs caso_id analizado justo antes (si son diferentes hay cambio de caso)
        cas_per[get_index(cas_per,cas_ant)] = casperant+'_FIN'   # Si no se ha cerrado el registro, se cierra el último caso registrado

    if caso_id_ not in new_cas:                  # Si el caso_id no está en new_cas es porque es un nuevo caso
      new_cas.append(caso_id_)                   # Se registra el nuevo caso
      if no_homicidio:
        cas_per.append(caso_id_+'_'+pers_id_vin)   # Se abre el caso + sus personas vinculadas
      else:
        cas_per.append(caso_id_)
    else:
      if '_FIN' in cas_per[get_index(cas_per,caso_id_)]:   # Si el caso sí está, se verifica si ya se terminó de registrar.  Si el caso ya se registró
        indx_to_drop.append(i)                             # por completo entonces esta instancia es redundante/antigua y se eliminará al final
      else:
        if no_homicidio:
          if pers_id_vin not in cas_per[get_index(cas_per,caso_id_)]:
            cas_per[get_index(cas_per,caso_id_)] = cas_per[get_index(cas_per,caso_id_)]+'_'+pers_id_vin
          else:                                    # Si ya fue registrada una persona con un tipo de vinculación, no se debería repetir dicha info
            indx_to_drop.append(i)                 # ser victima/indiciado/denunciante dos veces no debería ser correcto (pj: podría duplicarse la modalidad)
        else:
          indx_to_drop.append(i)

  
  df__ = df_.drop(indx_to_drop, inplace=False)             # Borrar todos los index que son duplicados antiguos
  df__.reset_index(drop=True, inplace=True)

  return df__,cas_per,new_cas

def save_interlost(df_original,df_depurado):
  '''
  Se genera un dataframe que guarda todos los intervinientes perdidos en un proceso
  de depuración. En caso de que no se reporten intervinientes en la última 
  actualización del caso_id, éstos serán removidos.

  Inputs.
  df_original: Dataframe que se depuró para mantener sólo la última actualización del caso
  df_depurado: Dataframe depurado con únicamente la última actualización del caso

  Output:
  df: Dataframe que contiene los intervinientes eliminados
  '''
  per = []
  vinc = []
  pervin = []

  for pers_i in np.unique(df_original.pers_id):
    if pers_i not in np.unique(df_depurado.pers_id):
      per.append(pers_i)
      vin = np.unique(df_original.loc[(df_original['pers_id']==pers_i)].tipo_vinculacion)[0]
      vinc.append(vin)
      pervin.append(str(pers_i)+'_'+str(vin))
  
  datos = {'pers_id': per, 'tipo_vinculacion': vinc, 'pers_id_vinculacion': pervin}
  df = pd.DataFrame(datos)
  return df

# Procesamiento
```
Se hará el procesamiento para:
1) Eliminar reportes antiguos de un mismo caso, es decir, mantener la última información del caso.

NOTA: En caso de que no se reporten intervinientes en la última actualización del caso_id, éstos serán removidos.
```



In [4]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [5]:
import pandas as pd
from glob import glob
import numpy as np

from shapely.geometry import Point
from shapely.geometry import shape
from tqdm import tqdm
import json

In [106]:
# df_fis = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/DELITOS_INF_FISCALIA_ACTUALIZADO_SOMECHANGE.xlsx') # Sin hur/hom/algunos fuera de pereira 151k +f
# df_fisdepu = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/datos_FIS_UltActCaso.xlsx')         # Último registro con COM_BARR_CUA y de 2020+ - 27.8k +f

# df_hur = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/HURTO_CON_INTER_PROCESSED_AREAS.xlsx')  # Hurto original con 166k +filas
# df_hurdepu = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/HURTO_CON_INTER_UltActCaso.xlsx')   # Hurtos solo última actualizacion 22.8k +filas

df_hom = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/HOMICIDIOS.xlsx')                         # Homicidios original con 1460 filas
df_homedepu = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/HOMICIDIOS_UltActCaso.xlsx')         # Homicidios solo última actualizacion con 327 filas

In [ ]:
# df_homdepu,cas_per,new_cas = get_ult_act_cas(df_hom, no_homicidio=False)
# df_homdepu

In [107]:
len(df_fis), len(df_fisdepu), len(df_hur), len(df_hurdepu), len(df_hom), len(df_homdepu)

(144126, 27806, 166843, 22892, 1460, 327)

In [108]:
len(np.unique(df_fis.caso_id)), len(np.unique(df_fisdepu.caso_id))

(10593, 10592)

In [109]:
len(np.unique(df_hur.caso_id)), len(np.unique(df_hurdepu.caso_id))

(12258, 12258)

In [110]:
len(np.unique(df_hom.caso_id)), len(np.unique(df_homdepu.caso_id))

(327, 327)

In [12]:
# ind = df_fis.loc[(df_fis['año_hecho']<2020)].index

# df_fis.drop(ind,inplace=True)             # Borrar todos los index que son duplicados antiguos
# df_fis.reset_index(drop=True,inplace=True)

In [ ]:
# df.drop(['zona_hechos','barrio_hechos'], axis = 'columns', inplace=True)

In [ ]:
for caso_id in np.unique(df_hur.caso_id):
  if caso_id not in np.unique(df_hurdepu.caso_id):
    print(caso_id)

In [43]:
df = save_interlost(df_hur,df_hurdepu)

In [44]:
len(df)

1541

In [45]:
df.tipo_vinculacion.value_counts()

DENUNCIANTE    1085
VICTIMA         274
INDICIADO       182
Name: tipo_vinculacion, dtype: int64

In [ ]:
for año in [2020,2021,2022,2023]:
  print(año)
  print(len(np.unique(df.loc[(df['año_hecho']== año)].caso_id)))
  print(len(np.unique(df_depu.loc[(df_depu['año_hecho']== año)].caso_id)))

In [ ]:
for año in [2020,2021,2022,2023]:
  print(año)
  for caso_id in np.unique(df.loc[(df['año_hecho']== año)].caso_id):
    if caso_id not in np.unique(df_depu.loc[(df_depu['año_hecho']== año)].caso_id):
      print(caso_id)
      print(df.loc[(df['caso_id']==caso_id)].fecha_hecho)
      print(df_depu.loc[(df_depu['caso_id']==caso_id)].fecha_hecho)
      print('=='*15)

In [ ]:
for año in [2020,2021,2022,2023]:
  print(año)
  for pers_id in np.unique(df.loc[(df['año_hecho']== año) & (df['tipo_vinculacion']== 'VICTIMA')].pers_id):
    if pers_id not in np.unique(df_depu.loc[(df_depu['año_hecho']== año) & (df_depu['tipo_vinculacion']== 'VICTIMA')].pers_id):
      print(pers_id)
      print(df.loc[(df['año_hecho']== año) & (df['pers_id']==pers_id) & (df['tipo_vinculacion']== 'VICTIMA')].fecha_hecho)
      print(df_depu.loc[(df_depu['año_hecho']== año) & (df_depu['pers_id']==pers_id) & (df_depu['tipo_vinculacion']== 'VICTIMA')].fecha_hecho)
      print('=='*15)

In [ ]:
df.fecha_hecho

0       2020-10-25
1       2020-10-25
2       2021-11-16
3       2021-11-16
4       2021-11-16
           ...    
29236   2022-11-25
29237   2022-11-25
29238   2022-11-25
29239   2023-01-07
29240   2023-01-07
Name: fecha_hecho, Length: 29241, dtype: datetime64[ns]

In [ ]:
df.fecha_hecho.apply(type)

0        <class 'pandas._libs.tslibs.timestamps.Timesta...
1        <class 'pandas._libs.tslibs.timestamps.Timesta...
2        <class 'pandas._libs.tslibs.timestamps.Timesta...
3        <class 'pandas._libs.tslibs.timestamps.Timesta...
4        <class 'pandas._libs.tslibs.timestamps.Timesta...
                               ...                        
29236    <class 'pandas._libs.tslibs.timestamps.Timesta...
29237    <class 'pandas._libs.tslibs.timestamps.Timesta...
29238    <class 'pandas._libs.tslibs.timestamps.Timesta...
29239    <class 'pandas._libs.tslibs.timestamps.Timesta...
29240    <class 'pandas._libs.tslibs.timestamps.Timesta...
Name: fecha_hecho, Length: 29241, dtype: object

In [ ]:
df_fis_depu['hora_hecho'] = df_fis_depu.hora_hecho.apply(str_to_hour)

In [ ]:
df_fis_depu['mes_hecho'] = df_fis_depu.fecha_hecho.apply(getMonth)
df_fis_depu['dia_hecho'] = df_fis_depu.fecha_hecho.apply(getDay)
df_fis_depu['año_hecho'] = df_fis_depu.fecha_hecho.apply(getYear)
df_fis_depu['hora_aprox_hecho'] = df_fis_depu.hora_hecho.apply(getHour_2)
df_fis_depu['semana_hecho'] = df_fis_depu.fecha_hecho.apply(getWeek)
df_fis_depu['dia_mes_hecho'] = df_fis_depu.fecha_hecho.apply(getDay)
df_fis_depu['dia_semana_hecho'] = df_fis_depu.fecha_hecho.apply(getWeek_Day_Name)
df_fis_depu['semana_total'] = get_semana_total(df_fis_depu)
df_fis_depu['x'] = df_fis_depu.longitud_hechos.apply(str_comma_to_float)
df_fis_depu['y'] = df_fis_depu.latitud_hechos.apply(str_comma_to_float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29241 entries, 0 to 29240
Data columns (total 88 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   caso_id                   29241 non-null  int64         
 1   año_corrido               29241 non-null  object        
 2   estado                    29240 non-null  object        
 3   etapa                     29240 non-null  object        
 4   hora_hecho                29241 non-null  datetime64[ns]
 5   municipio_hecho           29241 non-null  object        
 6   departamento_hecho        29241 non-null  object        
 7   codigo_dane_mun_hechos    29241 non-null  int64         
 8   region                    19488 non-null  object        
 9   latitud_hechos            29241 non-null  float64       
 10  longitud_hechos           29241 non-null  float64       
 11  seccional_asignacion      173 non-null    object        
 12  seccional_mapa_jud

In [ ]:
df.hora_hecho.isnull().sum()

0

In [ ]:
df.hora_hecho.apply(type)

0        <class 'pandas._libs.tslibs.timestamps.Timesta...
1        <class 'pandas._libs.tslibs.timestamps.Timesta...
2        <class 'pandas._libs.tslibs.timestamps.Timesta...
3        <class 'pandas._libs.tslibs.timestamps.Timesta...
4        <class 'pandas._libs.tslibs.timestamps.Timesta...
                               ...                        
29236    <class 'pandas._libs.tslibs.timestamps.Timesta...
29237    <class 'pandas._libs.tslibs.timestamps.Timesta...
29238    <class 'pandas._libs.tslibs.timestamps.Timesta...
29239    <class 'pandas._libs.tslibs.timestamps.Timesta...
29240    <class 'pandas._libs.tslibs.timestamps.Timesta...
Name: hora_hecho, Length: 29241, dtype: object

In [ ]:
df['mes_hecho'] = df.fecha_hecho.apply(getMonth)
df['dia_hecho'] = df.fecha_hecho.apply(getDay)
df['año_hecho'] = df.fecha_hecho.apply(getYear)
df['hora_aprox_hecho'] = df.hora_hecho.apply(getHour_2)
df['semana_hecho'] = df.fecha_hecho.apply(getWeek)
df['dia_mes_hecho'] = df.fecha_hecho.apply(getDay)
df['dia_semana_hecho'] = df.fecha_hecho.apply(getWeek_Day_Name)
df['semana_total'] = get_semana_total(df)
df['x'] = df.longitud_hechos.apply(str_comma_to_float)
df['y'] = df.latitud_hechos.apply(str_comma_to_float)

In [ ]:
len(np.unique(df.loc[(df['año_hecho']==2022)].caso_id))

4043

### GENERACIÓN DE TABLAS DELITOS + INTERVINIENTES

In [ ]:
df_ref = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/DELITOS/Processed/Semana 50_2022 - Processed/DELITOS DSX_CON_INT_ZonaPER.xlsx')

In [ ]:
ruta_inter = glob('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/Semana 2_2023/DIRECCIÓN SECCIONAL DE RISARALDA HURTO INTERVINIENTES.xlsx')[0]
ruta_del = glob('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/Semana 2_2023/DIRECCIÓN SECCIONAL DE RISARALDA HURTO.xlsx')[0]

In [ ]:
df_del = pd.read_excel(ruta_del)
df_inter = pd.read_excel(ruta_inter)

In [ ]:
len(df_del), len(df_inter)

(18444, 41299)

In [ ]:
for column in df_inter.columns:            # Buscar en qué columnas hay casos nulos o NaN, en casos como caso_id se elimina esa fila
  if df_inter[column].isna().sum() != 0:
    print(column, df_inter[column].isna().sum(),np.abs(np.where(df_inter[column].isna())))
  else:
    print(column, df_inter[column].isna().sum())

caso_id 0
año_corrido 0
delito 0
fecha_entrada 0
fecha_hecho 0
seccional_hechos 0
archivo 0
año_hecho 0
año_entrada 0
grupo 0
deli_id 0
tipo_vinculacion 0
pers_id 0
genero 2326 [[   30    56    68 ... 41269 41270 41282]]
edad 6911 [[   30    31    56 ... 41269 41270 41282]]
indigena 0
aplica_lgbti 0
defensor_ddhh 0
afrodescendiente 0
grupo_etario 0
nna 0
adulto_mayor 0
mujer 0
subcategoria_fp 41286 [[    0     1     2 ... 41296 41297 41298]]
funcionario 0
exfuncionario 0
cuenta_victima 22997 [[    0     2     4 ... 41293 41295 41297]]


In [ ]:
len(df_inter), df_inter.index

(41299, RangeIndex(start=0, stop=41299, step=1))

In [ ]:
# df_inter.drop([1947, 1948, 1949, 1950, 1951, 1952],inplace=True)         # Borrar index donde pueda generar problema (como caso_id NaN)
# df_inter.reset_index(drop=True,inplace=True)     

In [ ]:
len(df_inter), df_inter.index

(41299, RangeIndex(start=0, stop=41299, step=1))

In [ ]:
colu = []

for col in df_inter.columns:        # Verificar las columnas que están repetidas en intervinientes y en hurto
  if col in df_del.columns:       # También se verifica que los tipos de dato sean iguales para posteriormente comparar
    print(col ,  "-"*10,type(df_inter[str(col)][0]), "---- delito:", type(df_del[str(col)][0]))
    colu.append(col)
colu.remove('caso_id')
  # else:
  #   if col not in columns_ref:
  #     print('='*15,'NO debería existir: ', col)

caso_id ---------- <class 'numpy.int64'> ---- delito: <class 'numpy.int64'>


In [ ]:
# df_del.año_hecho = df_del.año_hecho.apply(int)

In [ ]:
# for col in df_inter.columns:                    # Para todas las columnas repetidas, se compara cada posición repetida 
#   if col in df_del:                           # En caso de que sean diferentes saldrá notificación
#     print(col)                                  # La idea es eliminar las columnas de la tabla de intervinientes verificando
#     for i in range(len(df_inter[str(col)])):    #  que no haya casos con datos diferentes. Así, esas columnas serán traídas
#       value_in = df_inter[str(col)].iloc[i]     #  desde la tabla de hurtos.
#       if value_in in df_del[str(col)].to_list():
#         indx_hur = df_del.loc[df_del[str(col)] == value_in].index[0]
#         if str(df_inter[str(col)].iloc[i]) != str(df_del[str(col)].iloc[indx_hur]):
#           print('Diferencia', col, str(df_inter[str(col)].iloc[i]), str(df_del[str(col)].iloc[indx_hur]))

In [ ]:
df_inter.drop(colu,axis=1, inplace=True)  # Eliminar columnas repetidas

In [ ]:
i=0
print(f'Delitos tiene en total {len(df_del.columns)} columnas')
print(f'Intervinientes tiene en total {len(df_inter.columns)} columnas')
print(f'Al final, Intervinientes quedará con {len(df_inter.columns) + len(df_del.columns)-1}')
print(f'Hay {len([col for col in df_del.columns if col not in df_inter])} columnas que se sumarán a intervinientes (Excepto caso_id)')

for col in df_del.columns:
  if col not in df_inter:
    df_inter[str(col)] = ""
print('\nAhora:')
print(f'Delitos tiene en total {len(df_del.columns)} columnas')
print(f'Intervinientes tiene en total {len(df_inter.columns)} columnas')

Delitos tiene en total 38 columnas
Intervinientes tiene en total 19 columnas
Al final, Intervinientes quedará con 56
Hay 37 columnas que se sumarán a intervinientes (Excepto caso_id)

Ahora:
Delitos tiene en total 38 columnas
Intervinientes tiene en total 56 columnas


In [ ]:
ind_to_drop = []

# def intervi_preprocess(df_hur, df_int):

for i in range(len(df_inter)):
  caso_id_inter = df_inter.caso_id.iloc[i]
  
  if caso_id_inter in df_del.caso_id.to_list():
    indx_hur = df_del.loc[df_del['caso_id'] == caso_id_inter].index[0]
    for col in df_del.columns:
      df_inter[str(col)].iloc[i] = df_del[str(col)].iloc[indx_hur]
  else:
    ind_to_drop.append(i) 

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
print(len(ind_to_drop), ind_to_drop)

0 []


In [ ]:
len(df_inter), df_inter.index

(41299, RangeIndex(start=0, stop=41299, step=1))

In [ ]:
df_inter.drop(ind_to_drop,inplace=True)
df_inter.reset_index(drop=True,inplace=True)

In [ ]:
len(df_inter), df_inter.index

(41299, RangeIndex(start=0, stop=41299, step=1))

In [ ]:
for col in df_inter.columns:
  try:
    unicos = np.unique(df_inter[str(col)])
    if len(unicos) < 15:
      print(col, len(unicos), unicos)
  except:
    unicos = [0]


archivo 2 ['NO' 'SI']
tipo_vinculacion 3 ['DENUNCIANTE' 'INDICIADO' 'VICTIMA']
indigena 2 ['NO' 'SI']
aplica_lgbti 2 ['NO' 'SI']
defensor_ddhh 2 ['NO' 'SI']
afrodescendiente 2 ['NO' 'SI']
grupo_etario 6 ['Adolescente de 14 a 17 años.' 'Adulto entre 27 y 59 años.'
 'Adulto mayor. Personas igual o mayor a 60 años.'
 'Joven de 18 a 26 años.' 'Niño, Niña. Población de 0 a 13 años.'
 'SIN DATO']
nna 2 ['NO' 'SI']
adulto_mayor 2 ['NO' 'SI']
mujer 2 ['NO' 'SI']
funcionario 2 ['NO' 'SI']
exfuncionario 1 ['NO']
cuenta_victima 3 [ 0.  1. nan]
año_corrido 2 ['NO' 'SI']
caso_averiguacion 2 ['N' 'S']
conexo 2 ['NO' 'SI']
etapa 6 ['EJECUCIÓN DE PENAS' 'INDAGACIÓN' 'INVESTIGACIÓN' 'JUICIO' 'QUERELLABLE'
 'TERMINACIÓN ANTICIPADA']
municipio_hecho 14 ['APÍA' 'BALBOA' 'BELÉN DE UMBRÍA' 'DOSQUEBRADAS' 'GUÁTICA' 'LA CELIA'
 'LA VIRGINIA' 'MARSELLA' 'MISTRATÓ' 'PEREIRA' 'PUEBLO RICO' 'QUINCHÍA'
 'SANTA ROSA DE CABAL' 'SANTUARIO']
departamento_hecho 1 ['Risaralda']
codigo_dane_mun_hechos 14 [66001.0 66045.0

In [ ]:
df_inter['mes_hecho'] = df_inter.fecha_hecho.apply(getMonth)
df_inter['dia_hecho'] = df_inter.fecha_hecho.apply(getDay)
df_inter['hora_aprox_hecho'] = df_inter.hora_hecho.apply(getHour)
df_inter['semana_hecho'] = df_inter.fecha_hecho.apply(getWeek)
df_inter['dia_mes_hecho'] = df_inter.fecha_hecho.apply(getDay)
df_inter['dia_semana_hecho'] = df_inter.fecha_hecho.apply(getWeek_Day_Name)
df_inter['semana_total'] = get_semana_total(df_inter)
df_inter['x'] = df_inter.longitud_hechos.apply(str_comma_to_float)
df_inter['y'] = df_inter.latitud_hechos.apply(str_comma_to_float)
df_inter['ObjectId2'] = [id for id in range(1, len(df_inter)+1)]

### Zonas Pereira

In [ ]:
with open('/content/drive/MyDrive/MyArcGisFolder/Perímetros Pereira.geojson', "r") as geoFile:
        geo_pe = json.load(geoFile)

In [ ]:
df = outside_drop(geo_pe, df)

1120


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
len(df) , df.index

(29453, RangeIndex(start=0, stop=29453, step=1))

### dia_semana_hecho
```
'LUNES':0
'MARTES':1
'MIÉRCOLES':2
'JUEVES':3
'VIERNES':4
'SÁBADO':5
'DOMINGO':6
```

In [ ]:
dias = {'LUNES':0, 'MARTES':1, 'MIÉRCOLES':2, 'JUEVES':3, 'VIERNES':4, 'SÁBADO':5, 'DOMINGO':6}

In [ ]:
for index,dia in enumerate(df_inter.dia_semana_hecho):
  df_inter.dia_semana_hecho[index] = dias[dia]

<ipython-input-787-67b4bc0401e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inter.dia_semana_hecho[index] = dias[dia]


In [ ]:
np.unique(df_inter.dia_semana_hecho)

array([0, 1, 2, 3, 4, 5, 6], dtype=object)

### Pasar de str a bool 
adulto_mayor - mujer - indigena - aplica_lgbti - defensor_ddhh - afrodescendiente - nna 

In [ ]:
for col in ['adulto_mayor', 'mujer', 'indigena', 'aplica_lgbti', 'defensor_ddhh', 'afrodescendiente', 'nna']:
  df_inter[col] = df_inter[col].apply(str_to_bool)

In [ ]:
np.unique(df_inter.indigena)

array([0, 1])

### GUARDAR

In [ ]:
Datos_FISC_ARcg = ['caso_id', 'año_corrido', 'estado', 'etapa', 'hora_hecho', 'municipio_hecho', 'departamento_hecho',
                   'codigo_dane_mun_hechos', 'region', 'latitud_hechos', 'longitud_hechos', 'seccional_asignacion', 
                   'seccional_mapa_jud_hecho', 'delito', 'modo_delito', 'modalidad_delito', 'caracterizacion_delito',
                   'denuncia_linea', 'grupo', 'grado_delito', 'ley', 'localidad_hechos', 'barrio_hechos', 'zona_hechos',
                   'direccion_hechos', 'titulo', 'capitulo', 'fecha_entrada', 'fecha_hecho', 'fecha_creacion', 'seccional_hechos',
                   'Agrupador', 'Categoria', 'archivo_2020', 'archivo', 'año_hecho', 'año_entrada', 'año_creacion', 'delito_agrup', 
                   'deli_id','tipo_vinculacion', 'pers_id', 'imno_id', 'genero', 'edad', 'indigena', 'aplica_lgbti', 'defensor_ddhh',
                   'afrodescendiente', 'grupo_etario', 'nna', 'adulto_mayor', 'mujer', 'subcategoria_fp', 'funcionario', 'exfuncionario',
                   'cuenta_victima', 'tipo_delito', 'id_ult_actuacion_caso', 'h_violento', 'grupo_delito', 'rural_urbana', 'semanas_2020',
                   'semanas_2021', 'seccional_hechos_x', 'archivo_2021', 'archivo_x', 'seccional_hechos_y', 'archivo_y', 'nunc',
                   'entidad_receptora', 'unidad_receptora', 'sale', 'mes_hecho', 'dia_hecho', 'hora_aprox_hecho', 'semana_hecho',
                   'dia_mes_hecho', 'dia_semana_hecho', 'ObjectId', 'semana_total', 'x', 'y', 'ObjectId2', 'GlobalID', 'VINCULACION_2']

In [ ]:
for col in Datos_FISC_ARcg:
  if col not in df_inter.columns:
    df_inter[col] = ""

In [ ]:
df_inter = df_inter[Datos_FISC_ARcg]

In [ ]:
Datos_FISC_ARcg == df_inter.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [ ]:
df_inter.año_hecho

0        2019.0
1        2019.0
2        2019.0
3        2019.0
4        2019.0
          ...  
28443    2023.0
28444    2023.0
28445    2023.0
28446    2022.0
28447    2022.0
Name: año_hecho, Length: 28448, dtype: object

In [ ]:
df_inter.año_hecho = df_inter.año_hecho.apply(int)
df_inter.año_creacion = df_inter.año_creacion.apply(int)
df_inter.cuenta_victima = df_inter.cuenta_victima.apply(float)

In [ ]:
base_name = ruta_inter.split('/')[-1][:-5].replace('DIRECCIÓN SECCIONAL DE RISARALDA','')
base_name = 'DELITOS HURTO_CON_INT_ZonaPER'

print(base_name)

DELITOS HURTO_CON_INT_ZonaPER


In [105]:
# savepath = '/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/Semana 2_2023 - Processed/'
# base_name = 'datos_FIS_UltActCaso_COMUBARRCUAD_mas2020'
# base_name = 'DELITOS_INF_FISCALIA_ACTUALIZADO_SOMECHANGE'
# base_name = 'HURTO_CON_INTER_UltActCaso'
# base_name = 'Delitos_fis_INTER_PERDIDOS_DEPURACION_UltActCaso'
# base_name = 'HURTOS_INTER_PERDIDOS_DEPURACION_UltActCaso'
base_name = 'HOMICIDIOS_UltActCaso'

savepath = '/content/drive/MyDrive/MyArcGisFolder/'
df_homdepu.to_excel(savepath + base_name +'.xlsx', index=False)
# df.to_csv(savepath + base_name +'.csv', index=False)

### Desarrollo para HURTOS

In [ ]:
df_ref = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/HURTO/PROCESSED/HURTO_CON_INTER_2022_50_PROCESSED_AREAS.xlsx')
columns_ref = df_ref.columns
print(len(columns_ref))

72


In [ ]:
file_name = 'DELITOS HURTO_CON_INT_ZonaPER_2023_2'
ruta = glob('/*/*/*/MyArcGisFolder/*/HURTO/'+file_name+'.xlsx')
print(ruta)

['/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/HURTO/DELITOS HURTO_CON_INT_ZonaPER_2023_2.xlsx']


In [ ]:
df = pd.read_excel(ruta[0])
print(len(df), 'x', len(df.columns))

28448 x 86


#### Buscando BARRIO y COMUNA (y su ID)

In [ ]:
# Acceso a los archivos
with open('/content/drive/MyDrive/MyArcGisFolder/Observatorio SC/2021/datos/Datos_georeferenciacion/comunas.geojson', "r") as geoFile:
  geo_com = json.load(geoFile)

with open('/content/drive/MyDrive/MyArcGisFolder/Observatorio SC/2021/datos/Datos_georeferenciacion/barrios.geojson', "r") as geoFile:
  geo_bar = json.load(geoFile)
        
with open('/content/drive/MyDrive/MyArcGisFolder/Observatorio SC/2021/datos/Datos_georeferenciacion/cuadrantes.geojson', "r") as geoFile:
  geo_cuad = json.load(geoFile)


In [ ]:
info_comuna = df[['x','y']].apply(lambda x: fix_comuna_barrio(x,geo_com,codigos=['nombre','codcom']),axis=1)
info_barrio = df[['x','y']].apply(lambda x: fix_comuna_barrio(x,geo_bar,codigos=['NOMBRE','FID']),axis=1)
info_cuad   = df[['x','y']].apply(lambda x: fix_comuna_barrio(x,geo_cuad,codigos=['NRO_CUADRANTE','DESCRIPCION']),axis=1)

In [ ]:
df['COMUNA']= info_comuna.apply(lambda x: x[0])
df['COMUNA_ID']= (info_comuna.apply(lambda x: x[1])).apply(comuna_id)

df['BARRIO']= info_barrio.apply(lambda x: x[0])
df['BARRIO_ID']= info_barrio.apply(lambda x: x[1])

df['CUADRANTE']= info_cuad.apply(lambda x: x[0])
df['CUADRANTE_DESCRIPCION']= info_cuad.apply(lambda x: x[1])

In [ ]:
df[['COMUNA','COMUNA_ID','BARRIO','BARRIO_ID','CUADRANTE','CUADRANTE_DESCRIPCION']]

,COMUNA,COMUNA_ID,BARRIO,BARRIO_ID,CUADRANTE,CUADRANTE_DESCRIPCION
0,RURAL,,RURAL,RURAL,MEPERMNVCCC01E04000000001,ESTACION DE POLICIA CIUDADELA DEL CAFE CUAD. 001
1,RURAL,,RURAL,RURAL,MEPERMNVCCC01E04000000001,ESTACION DE POLICIA CIUDADELA DEL CAFE CUAD. 001
2,Del Café,18,RURAL,RURAL,MEPERMNVCCC01E04000000002,ESTACION DE POLICIA CIUDADELA DEL CAFE CUAD. 002
3,Del Café,18,RURAL,RURAL,MEPERMNVCCC01E04000000002,ESTACION DE POLICIA CIUDADELA DEL CAFE CUAD. 002
4,Del Café,18,RURAL,RURAL,MEPERMNVCCC01E04000000002,ESTACION DE POLICIA CIUDADELA DEL CAFE CUAD. 002
...,...,...,...,...,...,...
29236,RURAL,,RURAL,RURAL,MEPERMNVCCC01E08S02000001,SUBESTACION DE POLICIA LA FLORIDA CUAD. 001
29237,RURAL,,RURAL,RURAL,MEPERMNVCCC01E08S02000001,SUBESTACION DE POLICIA LA FLORIDA CUAD. 001
29238,RURAL,,RURAL,RURAL,MEPERMNVCCC01E08S02000001,SUBESTACION DE POLICIA LA FLORIDA CUAD. 001
29239,Centro,3,Sector Galeria Central,441,MEPERMNVCCC01E01C09000005,CAI LA LIBERTAD CUAD. 005


#### Generando columnas con orden adecuado

In [ ]:
# Columnas en orden para datos en tablero de Hurtos.
# Se codifica grupo_etario y dia_semana_hecho
# No se codifican variables de SI/NO

colu_hur = ['caso_id', 'deli_id', 'tipo_vinculacion', 'pers_id', 'imno_id',
       'genero', 'edad', 'indigena', 'aplica_lgbti', 'defensor_ddhh',
       'afrodescendiente', 'grupo_etario', 'nna', 'adulto_mayor', 'mujer',
       'subcategoria_fp', 'funcionario', 'exfuncionario', 'cuenta_victima',
       'año_corrido', 'estado', 'etapa', 'hora_hecho', 'municipio_hecho',
       'departamento_hecho', 'codigo_dane_mun_hechos', 'region',
       'latitud_hechos', 'longitud_hechos', 'seccional_asignacion',
       'seccional_mapa_jud_hecho', 'delito', 'tipo_delito', 'modo_delito',
       'modalidad_delito', 'caracterizacion_delito', 'grupo', 'denuncia_linea',
       'grado_delito', 'ley', 'localidad_hechos', 'barrio_hechos',
       'zona_hechos', 'direccion_hechos', 'titulo', 'capitulo',
       'fecha_entrada', 'fecha_hecho', 'fecha_creacion', 'seccional_hechos',
       'archivo', 'archivo_2020', 'año_entrada', 'año_creacion', 'año_hecho',
       'id_ult_actuacion_caso', 'mes_hecho', 'dia_hecho', 'hora_aprox_hecho',
       'semana_hecho', 'dia_mes_hecho', 'dia_semana_hecho', 'semana_total',
       'x', 'y', 'COMUNA', 'COMUNA_ID', 'BARRIO', 'BARRIO_ID',
       'CUADRANTE', 'CUADRANTE_DESCRIPCION']

df = df[colu_hur]

In [ ]:
print(len(df), 'x', len(df.columns))

28448 x 72


### Grupo etario
```
0 = 0 a 13años
1 = 14 a 17
2 = 18 a 26
3 = 27 a 59
4 = 60 o más
5 = SIN DATO
```

In [ ]:
len(np.unique(df.grupo_etario)), np.unique(df.grupo_etario)

(6, array(['Adolescente de 14 a 17 años.', 'Adulto entre 27 y 59 años.',
        'Adulto mayor. Personas igual o mayor a 60 años.',
        'Joven de 18 a 26 años.', 'Niño, Niña. Población de 0 a 13 años.',
        'SIN DATO'], dtype=object))

In [ ]:
rangos = {'Niño, Niña. Población de 0 a 13 años.':0, 
          'Adolescente de 14 a 17 años.':1, 
          'Joven de 18 a 26 años.':2, 
          'Adulto entre 27 y 59 años.':3, 
          'Adulto mayor. Personas igual o mayor a 60 años.':4,
          'SIN DATO':5}

In [ ]:
for index,ran in enumerate(df.grupo_etario):
  df.grupo_etario[index] = rangos[ran]

<ipython-input-59-dc7687c064f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.grupo_etario[index] = rangos[ran]


In [ ]:
np.unique(df.grupo_etario)

array([0, 1, 2, 3, 4, 5], dtype=object)

### Pasar de bool a str
adulto_mayor - mujer - indigena - aplica_lgbti - defensor_ddhh - afrodescendiente - nna 

In [ ]:
for col in ['adulto_mayor', 'mujer', 'indigena', 'aplica_lgbti', 'defensor_ddhh', 'afrodescendiente', 'nna']:
  df[col] = df[col].apply(bool_to_string)

In [ ]:
np.unique(df.indigena)

array(['NO', 'SI'], dtype=object)

### GUARDAR

In [ ]:
print(columns_ref == df.columns)
print(len(df), 'x', len(df.columns))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]
28448 x 72


In [ ]:
df.año_hecho

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
28443    2023
28444    2023
28445    2023
28446    2022
28447    2022
Name: año_hecho, Length: 28448, dtype: int64

In [ ]:
df.año_hecho = df.año_hecho.apply(int)
df.año_creacion = df.año_creacion.apply(int)
df.cuenta_victima = df.cuenta_victima.apply(float)

In [ ]:
base_name = ruta[0].split('/')[-1][:-5]
base_name = 'HURTO_CON_INTER_2023_2_PROCESSED_AREAS'

print(base_name)

HURTO_CON_INTER_2023_2_PROCESSED_AREAS


In [ ]:
savepath = '/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/HURTO/PROCESSED/'
df.to_excel(savepath + base_name +'.xlsx', index=False)

## Verificación Datos Repetidos

In [ ]:
df_8 = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/HURTO/ORIGINALES/DIRECCIÓN SECCIONAL DE RISARALDA HURTO_2022_8.xlsx')
df_50 = pd.read_excel('/content/drive/MyDrive/MyArcGisFolder/Procesar x Delito/HURTO/ORIGINALES/DIRECCIÓN SECCIONAL DE RISARALDA HURTO_2022_50.xlsx')


In [ ]:
len(df_8), len(df_50)

(6989, 17883)

In [ ]:
len(df_8.columns)

40

In [ ]:
df_8.latitud_hechos  = df_8.latitud_hechos.apply(str_comma_to_float)

In [ ]:
for col in df_8.columns:        # Verificar las columnas que están repetidas en intervinientes y en hurto
  if col in df_50.columns:       # También se verifica que los tipos de dato sean iguales para posteriormente comparar
    print(col ,  "-"*10,type(df_8[str(col)][0]), "---- 50:", type(df_50[str(col)][0]))


caso_id ---------- <class 'numpy.int64'> ---- 50: <class 'numpy.int64'>
año_corrido ---------- <class 'str'> ---- 50: <class 'str'>
caso_averiguacion ---------- <class 'str'> ---- 50: <class 'str'>
estado ---------- <class 'str'> ---- 50: <class 'str'>
etapa ---------- <class 'str'> ---- 50: <class 'str'>
hora_hecho ---------- <class 'str'> ---- 50: <class 'str'>
municipio_hecho ---------- <class 'str'> ---- 50: <class 'str'>
departamento_hecho ---------- <class 'str'> ---- 50: <class 'str'>
codigo_dane_mun_hechos ---------- <class 'numpy.float64'> ---- 50: <class 'numpy.float64'>
region ---------- <class 'str'> ---- 50: <class 'str'>
latitud_hechos ---------- <class 'numpy.float64'> ---- 50: <class 'numpy.float64'>
longitud_hechos ---------- <class 'numpy.float64'> ---- 50: <class 'numpy.float64'>
seccional_asignacion ---------- <class 'str'> ---- 50: <class 'str'>
seccional_mapa_jud_hecho ---------- <class 'str'> ---- 50: <class 'str'>
tipo_delito ---------- <class 'str'> ---- 50: <c

In [ ]:
ind_8 = []
ind_50 = []

for indx_8,caso in enumerate(df_8.caso_id):
  if caso in df_50.caso_id.to_list():
    indx_50 = df_50.loc[df_50['caso_id'] == caso].index[0]
    ind_8.append(indx_8)
    ind_50.append(indx_50)

In [ ]:
len(ind_8), len(ind_50)

(6966, 6966)

In [ ]:
df_casos8 = df_8.iloc[ind_8].reset_index(drop=True)
df_casos50 = df_50.iloc[ind_50].reset_index(drop=True)

In [ ]:
for col in df_casos8.columns:
  rep = 0
  if col in df_casos50.columns:
    
    for i,valor in enumerate(df_casos8[str(col)]):
      if valor == df_casos50[str(col)].iloc[i]:
        rep += 1
      else:
        print(valor, '/// ',df_casos50[str(col)].iloc[i])
  print(col, rep)
    

caso_id 238
790535984 ///  790535801
790752999 ///  790752838
790753001 ///  790752838
790763593 ///  790763591
790873639 ///  790873515
791690751 ///  791690750
791923900 ///  791923852
792024334 ///  792024332
792216687 ///  790653850
792510311 ///  792510246
790727595 ///  790727578
791732136 ///  794031335
791762364 ///  791762168
106197500 ///  106197486
791989904 ///  791989862
106417569 ///  106417402
791428210 ///  791428204
792293071 ///  792288534
pers_id 220
42978848 ///  42978849
43233538 ///  43233540
43233545 ///  43233540
43245386 ///  43245381
43375513 ///  43375514
44362298 ///  44362297
44648876 ///  44648798
44767908 ///  44767906
45000358 ///  48348063
45356844 ///  45356828
43204734 ///  43204701
46016352 ///  47238556
44450783 ///  44450632
42354586 ///  42354571
44726383 ///  44726278
42609849 ///  42609729
44042701 ///  44042689
45092581 ///  45087581
imno_id 220
F ///  M
F ///  M
F ///  M
M ///  F
genero 234
NO ///  SI
indigena 237
aplica_lgbti 238
defensor_ddh